# Import libraries

Import Numpy and cv2 to enable using OpenCV.

In [1]:
import numpy as np
import cv2

# Define the functions

Define the function to get the background frame used for frame differencing and background subtraction. Use it to initialize the setting and to detect cars going to the city centre.

In [2]:
def get_background(file_path):
    """
    Get the median frame as the background frame.
    
    Parameters
    ----------
    file_path : str
    The file path in string.
    
    Return
    ----------
    median_frame : NumPy array
    The median frame.
    
    """   
    cap = cv2.VideoCapture(file_path)
    
    # randomly select 50 frames for the calculating the median
    frame_indices = cap.get(cv2.CAP_PROP_FRAME_COUNT) * np.random.uniform(size=50)
    
    # store the frames in array
    frames = []
    for idx in frame_indices:
        # set the frame id to read that particular frame
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        frames.append(frame)

    # calculate the median
    median_frame = np.median(frames, axis=0).astype(np.uint8)
    
    return median_frame


def initialize(file_path):
    """
    Initialize the setting. 
    Get the background frame and read the video file.
    
    Parameters
    ----------
    file_path : str
    The file path in string.
    
    Return
    ----------
    background, cap : NumPy array, cv2.VideoCapture
    The background frame and the video capture.
    
    """       
    # get the background frame
    background = get_background(file_path)

    # convert the background model to grayscale format
    background = cv2.cvtColor(background, cv2.COLOR_BGR2GRAY)
    
    # Read the video file
    cap = cv2.VideoCapture(file_path)

    return background, cap


def detect_cars_going_to_cityCentre(background, cap, consecutive_frame):
    """
    Detect cars going to the city centre and count them.
    Print the detection history and the count result.
    
    Parameters
    ----------
    background : NumPy array
    The background frame.

    cap : cv2.VideoCapture
    The video capture.
    
    consecutive_frame : int
    The number of consecutive frame per process.
    
    """       
    frame_count = 0
    car_count = 0
    frame_diff_list = []

    # get duration of the video in second
    fps = cap.get(cv2.CAP_PROP_FPS) 
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = frame_count/fps

    while cap.isOpened():
        ret, frame = cap.read()

        if ret == True:       

            frame_count += 1
            orig_frame = frame.copy()        

            # convert the frame to grayscale
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # initialize frame difference list 
            if frame_count == 1 or frame_count % consecutive_frame == 0:
                frame_diff_list = []

            # find the difference between current frame and base frame
            frame_diff = cv2.absdiff(gray, background)

            # thresholding to convert the frame to binary
            ret, thres = cv2.threshold(frame_diff, 50, 255, cv2.THRESH_BINARY)

            # dilate the frame a bit to get some more white area...
            # ... makes the detection of contours a bit easier
            dilate_frame = cv2.dilate(thres, None, iterations=2)


            # append the final result into the `frame_diff_list`
            frame_diff_list.append(dilate_frame)

            # if we have reached `consecutive_frame` number of frames
            if len(frame_diff_list) == consecutive_frame:

                # add all the frames in the `frame_diff_list`
                sum_frames = sum(frame_diff_list)

                # find the contours around the white segmented areas
                contours, hierarchy = cv2.findContours(sum_frames, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)            

                for contour in contours:   

                    # get coordinates from the contour
                    (x, y, w, h) = cv2.boundingRect(contour)
                    
                    # count if it's car and is passing by the detection area
                    if (x >= 460) and (x <= 490) and (y >= 350) and (y <= 420) and cv2.contourArea(contour) >= 3000:                    
                        car_count+=1
                        print('detected', car_count)
                        # draw the bounding boxes
                        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)               

            # show frame that indicates detection area
            cv2.rectangle(frame, (460, 350), (490, 420), (0, 255, 0), 2) 
            cv2.imshow('video', frame)          

            # stop the program by pressing 'q'   
            if cv2.waitKey(1) == ord('q'):
                break
        else:
            break

    # destroy all the windows after the loop release the video object
    cap.release()
    cv2.destroyAllWindows()
    
    # Print the results
    print('Total number of cars: ', car_count)
    print('Duration in second: ', duration)
    print('Cars per minute: ', int((car_count / duration) * 60))


# Detect cars going to the city centre

Print the history of detections, the total number of cars, duration in second, and calculate the number of cars per minute.

## Video 1

In [3]:
background, cap = initialize('Traffic_Laramie_1.mp4')

# the number of consecutive frames per process 
# set 5 as car speed is slower
consecutive_frame = 5

detect_cars_going_to_cityCentre(background, cap, consecutive_frame)

detected 1
detected 2
detected 3
detected 4
detected 5
detected 6
Total number of cars:  6
Duration in second:  177.92
Cars per minute:  2


## Video 2

In [4]:
background, cap = initialize('Traffic_Laramie_2.mp4')

# set 4 as car speed is faster 
consecutive_frame = 4

detect_cars_going_to_cityCentre(background, cap, consecutive_frame)

detected 1
detected 2
detected 3
detected 4
Total number of cars:  4
Duration in second:  105.68
Cars per minute:  2
